# Chapter 2. A Quick Tour of Traditional NLP

Natural language processing (NLP, introduced in the previous chapter) and computational linguistics (CL) are two areas of computational study of human language. NLP aims to develop methods for solving practical problems involving language, such as information extraction, automatic speech recognition, machine translation, sentiment analysis, question answering, and summarization. CL, on the other hand, employs computational methods to understand properties of human language. How do we understand language? How do we produce language? How do we learn languages? What relationships do languages have with one another?

In literature, it is common to see a crossover of methods and researchers, from CL to NLP and vice versa. Lessons from CL about language can be used to inform priors in NLP, and statistical and machine learning methods from NLP can be applied to answer questions CL seeks to answer. In fact, some of these questions have ballooned into disciplines of their own, like phonology, morphology, syntax, semantics, and pragmatics.

In this book, we concern ourselves with only NLP, but we borrow ideas routinely from CL as needed. Before we fully vest ourselves into neural network methods for NLP—the focus of the rest of this book—it is worthwhile to review some traditional NLP concepts and methods. That is the goal of this chapter.

If you have some background in NLP, you can skip this chapter, but you might as well stick around for nostalgia and to establish a shared vocabulary for the future.

## Corpora, Tokens, and Types
All NLP methods, be they classic or modern, begin with a text dataset, also called a corpus (plural: corpora). A corpus usually contains raw text (in ASCII or UTF-8) and any metadata associated with the text. The raw text is a sequence of characters (bytes), but most times it is useful to group those characters into contiguous units called tokens. In English, tokens correspond to words and numeric sequences separated by white-space characters or punctuation.

The metadata could be any auxiliary piece of information associated with the text, like identifiers, labels, and timestamps. In machine learning parlance, the text along with its metadata is called an instance or data point. The corpus (Figure 2-1), a collection of instances, is also known as a dataset. Given the heavy machine learning focus of this book, we freely interchange the terms corpus and dataset throughout.

<div align="center" style="width: 100%;">
    <div align="center" style="width: 600px">
        <img src="imgs/nlpp_0201.png">
        <h4 style="font-family: courier; font-size: .8em;">Figure 2-1. The corpus: the starting point of NLP tasks.</h4>
    </div>
</div>

The process of breaking a text down into tokens is called tokenization. For example, there are six tokens in the Esperanto sentence “Maria frapis la verda sorĉistino.”1 Tokenization can become more complicated than simply splitting text based on nonalphanumeric characters, as is demonstrated in Figure 2-2. For agglutinative languages like Turkish, splitting on whitespace and punctuation might not be sufficient, and more specialized techniques might be warranted. As you will see in Chapters 4 and 6, it may be possible to entirely circumvent the issue of tokenization in some neural network models by representing text as a stream of bytes; this becomes very important for agglutinative languages.

<div align="center" style="width: 100%;">
    <div align="center" style="width: 600px">
        <img src="imgs/nlpp_0202.png">
        <h4 style="font-family: courier; font-size: .8em;">Figure 2-2. Tokenization in languages like Turkish can become complicated quickly.</h4>
    </div>
</div>

Finally, consider the following tweet:

<div align="center" style="width: 100%;">
    <div align="center" style="width: 600px">
        <img src="imgs/nlpp_02_jesustweet.png">
    </div>
</div>

Tokenizing tweets involves preserving hashtags and @handles, and segmenting smilies such as :-) and URLs as one unit. Should the hashtag #MakeAMovieCold be one token or four? Most research papers don’t give much attention to these matters, and in fact, many of the tokenization decisions tend to be arbitrary—but those decisions can significantly affect accuracy in practice more than is acknowledged. Often considered the grunt work of preprocessing, most open source NLP packages provide reasonable support for tokenization to get you started. Example 2-1 shows examples from NLTK and spaCy, two commonly used packages for text processing.

In [16]:
import spacy

nlp = spacy.load('en_core_web_sm')

text = "Mary, don’t slap the green witch"
print([str(token) for token in nlp(text.lower())])

['mary', ',', 'do', 'n’t', 'slap', 'the', 'green', 'witch']


In [17]:
from nltk.tokenize import TweetTokenizer

tweet=u"Snow White and the Seven Degrees #MakeAMovieCold@midnight:-)"

tokenizer = TweetTokenizer()
print(tokenizer.tokenize(tweet.lower()))

['snow', 'white', 'and', 'the', 'seven', 'degrees', '#makeamoviecold', '@midnight', ':-)']


Types are unique tokens present in a corpus. The set of all types in a corpus is its vocabulary or lexicon. Words can be distinguished as content words and stopwords. Stopwords such as articles and prepositions serve mostly a grammatical purpose, like filler holding the content words.

> FEATURE ENGINEERING

> This process of understanding the linguistics of a language and applying it to solving NLP problems is called feature engineering. This is something that we keep to a minimum here, for convenience and portability of models across languages. But when building and deploying real-world production systems, feature engineering is indispensable, despite recent claims to the contrary. For an introduction to feature engineering in general, consider reading the book by Zheng and Casari (2016).

## Unigrams, Bigrams, Trigrams, …, N-grams
N-grams are fixed-length (n) consecutive token sequences occurring in the text. A bigram has two tokens, a unigram one. Generating n-grams from a text is straightforward enough, as illustrated in Example 2-2, but packages like spaCy and NLTK provide convenient methods.

In [20]:
def n_grams(text, n):
    '''
    takes tokens or text, returns a list of n-grams
    '''
    return [text[i:i+n] for i in range(len(text)-n+1)]

cleaned = ['mary', ',', "n't", 'slap', 'green', 'witch', '.']

n_grams(cleaned, 3)

[['mary', ',', "n't"],
 [',', "n't", 'slap'],
 ["n't", 'slap', 'green'],
 ['slap', 'green', 'witch'],
 ['green', 'witch', '.']]

For some situations in which the subword information itself carries useful information, one might want to generate character n-grams. For example, the suffix “-ol” in “methanol” indicates it is a kind of alcohol; if your task involved classifying organic compound names, you can see how the subword information captured by n-grams can be useful. In such cases, you can reuse the same code, but treat every character n-gram as a token.2

## Lemmas and Stems
Lemmas are root forms of words. Consider the verb fly. It can be inflected into many different words—flow, flew, flies, flown, flowing, and so on—and fly is the lemma for all of these seemingly different words. Sometimes, it might be useful to reduce the tokens to their lemmas to keep the dimensionality of the vector representation low. This reduction is called lemmatization, and you can see it in action in Example 2-3.

In [19]:
doc = nlp(u"he was running late")

for token in doc:
    print(f'{token} --> {token.lemma_}')

he --> he
was --> be
running --> run
late --> late


spaCy, for example, uses a predefined dictionary, called WordNet, for extracting lemmas, but lemmatization can be framed as a machine learning problem requiring an understanding of the morphology of the language.

Stemming is the poor-man’s lemmatization.3 It involves the use of handcrafted rules to strip endings of words to reduce them to a common form called stems. Popular stemmers often implemented in open source packages include the Porter and Snowball stemmers. We leave it to you to find the right spaCy/NLTK APIs to perform stemming.

## Categorizing Sentences and Documents

Categorizing or classifying documents is probably one of the earliest applications of NLP. The TF and TF-IDF representations we described in Chapter 1 are immediately useful for classifying and categorizing longer chunks of text such as documents or sentences. Problems such as assigning topic labels, predicting sentiment of reviews, filtering spam emails, language identification, and email triaging can be framed as supervised document classification problems. (Semi-supervised versions, in which only a small labeled dataset is used, are incredibly useful, but that topic is beyond the scope of this book.)

## Categorizing Words: POS Tagging

We can extend the concept of labeling from documents to individual words or tokens. A common example of categorizing words is part-of-speech (POS) tagging, as demonstrated in Example 2-4.

In [21]:
doc = nlp(u"Mary slapped the green witch.")

for token in doc:
    print(f'{token} - {token.pos_}')

Mary - PROPN
slapped - VERB
the - DET
green - ADJ
witch - NOUN
. - PUNCT


## Categorizing Spans: Chunking and Named Entity Recognition
Often, we need to label a span of text; that is, a contiguous multitoken boundary. For example, consider the sentence, “Mary slapped the green witch.” We might want to identify the noun phrases (NP) and verb phrases (VP) in it, as shown here:

> [NP Mary] [VP slapped] [the green witch].

This is called chunking or shallow parsing. Shallow parsing aims to derive higher-order units composed of the grammatical atoms, like nouns, verbs, adjectives, and so on. It is possible to write regular expressions over the part-of-speech tags to approximate shallow parsing if you do not have data to train models for shallow parsing. Fortunately, for English and most extensively spoken languages, such data and pretrained models exist. Example 2-5 presents an example of shallow parsing using spaCy.

In [22]:
doc  = nlp(u"Mary slapped the green witch.")

for chunk in doc.noun_chunks:
    print(f'{chunk} - {chunk.label_}')

Mary - NP
the green witch - NP


Another type of span that’s useful is the named entity. A named entity is a string mention of a real-world concept like a person, location, organization, drug name, and so on. Here’s an example:

![](https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/assets/nlpp_02_NER.png)

## Structure of Sentences
Whereas shallow parsing identifies phrasal units, the task of identifying the relationship between them is called parsing. You might recall from elementary English class diagramming sentences like in the example shown in Figure 2-3.

<div align="center" style="width: 100%;">
    <div align="center" style="width: 600px">
        <img src="imgs/nlpp_0203.png">
        <h4 style="font-family: courier; font-size: .8em;">Figure 2-3. A constituent parse of the sentence “Mary slapped the green witch.”</h4>
    </div>
</div>

Parse trees indicate how different grammatical units in a sentence are related hierarchically. The parse tree in Figure 2-3 shows what’s called a constituent parse. Another, possibly more useful, way to show relationships is using dependency parsing, depicted in Figure 2-4.

<div align="center" style="width: 100%;">
    <div align="center" style="width: 600px">
        <img src="imgs/nlpp_0204.png">
        <h4 style="font-family: courier; font-size: .8em;">Figure 2-4. A dependency parse of the sentence “Mary slapped the green witch.”</h4>
    </div>
</div>

To learn more about traditional parsing, see the “References” section at the end of this chapter.

## Word Senses and Semantics

Words have meanings, and often more than one. The different meanings of a word are called its senses. WordNet, a long-running lexical resource project from Princeton University, aims to catalog the senses of all (well, most) words in the English language, along with other lexical relationships.4 For example, consider a word like “plane.” Figure 2-5 shows the different senses in which this word could be used.

<div align="center" style="width: 100%;">
    <div align="center" style="width: 600px">
        <img src="imgs/nlpp_0205.png">
        <h4 style="font-family: courier; font-size: .8em;">Figure 2-5. Senses for the word “plane” (courtesy of WordNet).</h4>
    </div>
</div>

The decades of effort that have been put into projects like WordNet are worth availing yourself of, even in the presence of modern approaches. Later chapters in this book present examples of using existing linguistic resources in the context of neural networks and deep learning methods.

Word senses can also be induced from the context—automatic discovery of word senses from text was actually the first place semi-supervised learning was applied to NLP. Even though we don’t cover that in this book, we encourage you to read Jurafsky and Martin (2014), Chapter 17, and Manning and Schütze (1999), Chapter 7.

# Summary
In this chapter, we reviewed some basic terminology and ideas in NLP that should be handy in future chapters. This chapter covered only a smattering of what traditional NLP has to offer. We omitted significant aspects of traditional NLP because we want to allocate the bulk of this book to the use of deep learning for NLP. It is, however, important to know that there is a rich body of NLP research work that doesn’t use neural networks, and yet is highly impactful (i.e., used extensively in building production systems). The neural network–based approaches should be considered, in many cases, as a supplement and not a replacement for traditional methods. Experienced practitioners often use the best of both worlds to build state-of-the-art systems. To learn more about the traditional approaches to NLP, we recommend the references listed in the following section.

# References

1. **Manning, Christopher D., and Hinrich Schütze. (1999). Foundations of Statistical Natural Language Processing. MIT press.**
2. **Bird, Steven, Ewan Klein, and Edward Loper. (2009). Natural Language Processing with Python: Analyzing Text with the Natural Language Toolkit. O’Reilly.**
3. **Smith, Noah A. (2011). Linguistic Structure prediction. Morgan and Claypool.**
4. **Jurafsky, Dan, and James H. Martin. (2014). Speech and Language Processing, Vol. 3. Pearson.**
5. **Russell, Stuart J., and Peter Norvig. (2016). Artificial Intelligence: A Modern Approach. Pearson.**
6. **Zheng, Alice, and Casari, Amanda. (2018). Feature Engineering for Machine Learning: Principles and Techniques for Data Scientists. O’Reilly.**

---

1. Translation: “Mary slapped the green witch.” We use this sentence as a running example in this chapter. We acknowledge the example is rather violent, but our use is a hat-tip to the most famous artificial intelligence textbook of our times (Russell and Norvig, 2016), which also uses this sentence as a running example.
2. In Chapters 4 and 6, we look at deep learning models that implicitly capture this substructure efficiently.
3. To understand the difference between stemming and lemmatization, consider the word “geese.” Lemmatization produces “goose,” whereas stemming produces “gees.”
4. Attempts to create multilingual versions of WordNet exist. See BabelNet as an example.